<a href="https://colab.research.google.com/github/sominshim/Predicting_Personality_through_Text/blob/Ujin/mbti_prediction_with_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **준비 사항**

In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 53.8MB/s 
     |████████████████████████████████| 890kB 58.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b31d2efed83b4d0e5a5f7e7398d8f0a889068a1e9d2af15fb4a896946dc58db4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
from google.colab import files
myfile = files.upload()

Saving data_winter_final.csv to data_winter_final.csv


In [4]:
data = pd.read_csv("data_winter_final.csv")
data.head()

,type,posts
0,ESTP,1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
1,INFP,"제게 큰 현타를 주었던 글이에요.제 자신이 마음에 안 들고,늘 부족한 것 같고,절대..."
2,ISFJ,"유고슬라비아는 남슬라브 국가들(세르비아, 슬로베니아, 크로아티아, 보스니아, 북마케..."
3,ISTP,10대 여자고 키도 크고 나이에 비해 성숙한 이미지를 가지고 있어요. 잇팁답게 차갑...
4,ISFP,제가 요즘 끌리는 남자가있는데 esfp인거같아요esfj인줄 알았는데 상황에따라 행동...


In [5]:
data = data.drop_duplicates("posts", keep="first")
data.dropna(axis=0, inplace=True)

In [6]:
df = data.copy()
df = df[df.posts.str.len()>50]
df.reset_index(inplace=True)
df = df.drop(['index'],axis=1)
df.shape

(19935, 2)

In [12]:
data = df.copy()

## mbti라벨링

In [13]:
# Split mbti personality into 4 letters and binarize
titles = ["Extraversion (E) - Introversion (I)",
          "Sensation (S) - INtuition (N)",
          "Thinking (T) - Feeling (F)",
          "Judgement (J) - Perception (P)"
         ] 
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]


#transform mbti to binary vector
def translate_personality(personality):
    return [b_Pers[l] for l in personality]

#transform binary vector to mbti personality
def translate_back(personality):
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

list_personality_bin = np.array([translate_personality(p) for p in data.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

Binarize MBTI list: 
[[1 1 1 1]
 [0 0 0 1]
 [0 1 0 0]
 ...
 [0 0 0 0]
 [0 1 1 0]
 [0 1 0 1]]


In [14]:

data['I-E'] = list_personality_bin[:,0]
data['N-S'] = list_personality_bin[:,1]
data['F-T'] = list_personality_bin[:,2]
data['J-P'] = list_personality_bin[:,3]

In [15]:
data = data.reset_index(drop=True)


## train, test split

In [66]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data[['type']]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

strat_train_set = strat_train_set.reset_index(drop=True)
strat_test_set = strat_test_set.reset_index(drop=True)

X_train = strat_train_set.posts
X_test = strat_test_set.posts

print(X_train.shape)
print(X_test.shape)

(15948,)
(3987,)


In [67]:
X_train

0        다행히 여기서 털어놓으니까..어제는 하나도 그렇지 않았는데,또 악몽에 시달릴까봐 힘...
1        ISTJ인가요?저와 성격 완전 똑같은데 문득 궁금해서요ㅎ내향적이고 상대 감정따윈 상...
2        아드레날린 나올일이 없으니나쁜엔팁이 되어가는 즁입니다​뭐 재미있고 익사이팅한거 없나...
3        헉!!!! 아디 바꿨습니당 ㅎㅎㅎ한 2년간 계속 인티제 나와서 인티젠 줄 알았더니,...
4        도화살보다 치명적인게 홍염살이라니 뭔말인지 모르겠다둘다 똑같은거 아닌감요https:...
                               ...                        
15943    매일 규칙적으로 1시간씩 늦게 자는 나를 보면 감탄스러울 따름이다. 어젠 8시 엊그...
15944    “Mbti 유형별 동물” 이런 짤들 보면 인프피는 물개인 경우들이 있더라구요 (거북...
15945    실수를 내가하면 나에게는 책임이 따르고 이해를 덜해주려는건 왜그럴까요 분위기가 생긴...
15946    ​​​핸드폰에 뭘 잘 안 까는 타입인데 기본 인터넷으로만 하기 귀찮아서네이버 카페 ...
15947    항상 보면 다 검은색만 선호하는것 같아요​친구랑 찍은 사진만 봐도 옷과  가방이블랙...
Name: posts, Length: 15948, dtype: object

In [68]:
#y_train = data['I-E'][train_index].values
#y_test = data['I-E'][test_index].values

## 전처리

In [69]:
sentences = strat_train_set['posts']
sentences[:10]

0    다행히 여기서 털어놓으니까..어제는 하나도 그렇지 않았는데,또 악몽에 시달릴까봐 힘...
1    ISTJ인가요?저와 성격 완전 똑같은데 문득 궁금해서요ㅎ내향적이고 상대 감정따윈 상...
2    아드레날린 나올일이 없으니나쁜엔팁이 되어가는 즁입니다​뭐 재미있고 익사이팅한거 없나...
3    헉!!!! 아디 바꿨습니당 ㅎㅎㅎ한 2년간 계속 인티제 나와서 인티젠 줄 알았더니,...
4    도화살보다 치명적인게 홍염살이라니 뭔말인지 모르겠다둘다 똑같은거 아닌감요https:...
5    링크공유하면 링크통해서 유튜브 채널로 들어가져야 할텐데 유튜브영상이 게시판에서 바로...
6    독서실 가본적은 없지만 ㅋㅋㅋㅋ 왠지 독서실 느낌이라서너무 좋아요 💝오늘 이 친구랑...
7    문제가 쉽고 제가 약한 편인 문화 유산과 불교 문제가 안 나와서 수월하게 풀었네요....
8    제 주변에 MBTI 검사결과 ENTJ가 나온 사람은 저 뿐입니다.제가 제 자신이 타...
9    어떻게 하시나여사실 제 지금 상황인데원래 7시 반에 일어나려다가 졸려서 더 잣는데 ...
Name: posts, dtype: object

In [70]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 다행히 여기서 털어놓으니까..어제는 하나도 그렇지 않았는데,또 악몽에 시달릴까봐 힘드네요 벌써부터..\u200b악몽꾸면 힘든 나날이거든요..10년째 말이에요..아..힘들어요. [SEP]',
 '[CLS] ISTJ인가요?저와 성격 완전 똑같은데 문득 궁금해서요ㅎ내향적이고 상대 감정따윈 상관 안하고 팩폭ㅋ\u200bISTJ or INTJ둘중 하나일것 같은데 사건 해결할때 단서 하나하나 꼼꼼하게 놓치지 않고, 자신의 경험을 바탕으로 유추하는거 보면 N보다는 S에 더 가깝다고 생각되는데 어떤게 확실한지 궁금하네요ㅎ\u200b [SEP]',
 '[CLS] 아드레날린 나올일이 없으니나쁜엔팁이 되어가는 즁입니다\u200b뭐 재미있고 익사이팅한거 없나요??!😲😲😲 [SEP]',
 '[CLS] 헉!!!! 아디 바꿨습니당 ㅎㅎㅎ한 2년간 계속 인티제 나와서 인티젠 줄 알았더니,요새 한 3주간 계속 엔티제 나오네요. 이제 깔끔히 인정하고 잠시 엔티제로 갈아타려 합니다. 아이디도 변경했어요. ㅎㅎ제가 저 스스로를 인티제 별종이라고 자꾸 했더니만 ㅋㅋㅋ 그게 아니라 좀 달랐나봐요?뭐 다 중간중간하는 성격이라 나중에 또 바뀔지도 몰라요 나중엔 F도 한번 해보고 싶네요 내 사랑 인티제 잠시 안녕~! ㅎㅎ 애니어그램 3w2 카멜레온이라 자꾸 바뀌어요 ㅠㅡㅠ ㅋㅋ [SEP]',
 '[CLS] 도화살보다 치명적인게 홍염살이라니 뭔말인지 모르겠다둘다 똑같은거 아닌감요https://mydohwa.forceteller.com당신의 치명적 매력은? 당신에게 잠재된 도화살을 알아보세요 / 공식 포스텔러 도화테스트 / 도화살테스트 / 도화살 테스트 / 도화살 / 도화mydohwa.forceteller.com\u200b [SEP]',
 '[CLS] 링크공유하면 링크통해서 유튜브 채널로 들어가져야 할텐데 유튜브영상이 게시판에서 바로 재생돼버리네요; 유튜브영상 링크공유할때 어떻게들하시나요? 유튜버분꺼 바로재생안되게 주소만 올리는법좀 가르쳐주세요~ [SEP]',
 '[CLS] 독서실 가본적은 없

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력 형식입니다. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됩니다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 합니다. [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 합니다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣습니다.
<br>
<br>
<br>

In [71]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 다행히 여기서 털어놓으니까..어제는 하나도 그렇지 않았는데,또 악몽에 시달릴까봐 힘드네요 벌써부터..​악몽꾸면 힘든 나날이거든요..10년째 말이에요..아..힘들어요. [SEP]
['[CLS]', '다', '##행', '##히', '여기서', '털', '##어', '##놓', '##으', '##니', '##까', '.', '.', '어', '##제는', '하', '##나', '##도', '그', '##렇', '##지', '않', '##았', '##는데', ',', '또', '악', '##몽', '##에', '시', '##달', '##릴', '##까', '##봐', '힘', '##드', '##네', '##요', '벌', '##써', '##부터', '.', '.', '악', '##몽', '##꾸', '##면', '힘', '##든', '나', '##날', '##이', '##거', '##든', '##요', '.', '.', '10', '##년', '##째', '말', '##이', '##에', '##요', '.', '.', '아', '.', '.', '힘', '##들어', '##요', '.', '[SEP]']


BERT는 형태소분석으로 토큰을 분리하지 않습니다. WordPiece라는 통계적인 방식을 사용합니다. 한 단어내에서 자주 나오는 글자들을 붙여서 하나의 토큰으로 만듭니다. 이렇게 하면 언어에 상관없이 토큰을 생성할 수 있다는 장점이 있습니다. 또한 신조어 같이 사전에 없는 단어를 처리하기도 좋습니다. 

위의 결과에서 ## 기호는 앞 토큰과 이어진다는 표시입니다. 토크나이저는 여러 언어의 데이터를 기반으로 만든 'bert-base-multilingual-cased'를 사용합니다. 그래서 한글도 처리가 가능합니다.
<br>
<br>
<br>

In [72]:
# 라벨 추출
labels_IE = strat_train_set['I-E'].values
labels_NS = strat_train_set['N-S'].values
labels_FT = strat_train_set['F-T'].values
labels_JP = strat_train_set['J-P'].values

In [73]:
sum=0
max=0
min=1000000000000000
for i in range(len(tokenized_texts)):
  sum = sum + len(tokenized_texts[i])
  if max < len(tokenized_texts[i]) :
    max = len(tokenized_texts[i])
  elif min > len(tokenized_texts[i]) :
    min = len(tokenized_texts[i])
  print(i,end=' ')

print(sum)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [74]:
print(min,max,sum)

3 16850 4407341


In [75]:
sum / len(tokenized_texts[i])

77321.77192982456

In [76]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9056,  25549,  18108, 108954,   9864,  12965, 118748,
       119185,  25503, 118671,    119,    119,   9546,  87164,   9952,
        16439,  12092,   8924, 118871,  12508,   9523, 119118,  41850,
          117,   9144,   9520, 118941,  10530,   9485,  89851,  85836,
       118671, 118990,  10028,  15001,  77884,  48549,   9339,  73131,
        17655,    119,    119,   9520, 118941, 118694,  14867,  10028,
        90537,   8982,  41919,  10739,  41521,  90537,  48549,    119,
          119,  10150,  10954,  93119,   9251,  10739,  10530,  48549,
          119,    119,   9519,    119,    119,  10028,  93200,  48549,
          119,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [77]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [78]:
#훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels_IE, validation_labels_IE, train_labels_NS, validation_labels_NS, train_labels_FT, validation_labels_FT, train_labels_JP, validation_labels_JP= train_test_split(input_ids,labels_IE, labels_NS, labels_FT, labels_JP,
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)


# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels_IE = torch.tensor(train_labels_IE)
train_labels_NS = torch.tensor(train_labels_NS)
train_labels_FT = torch.tensor(train_labels_FT)
train_labels_JP = torch.tensor(train_labels_JP)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels_IE = torch.tensor(validation_labels_IE)
validation_labels_NS = torch.tensor(validation_labels_NS)
validation_labels_FT = torch.tensor(validation_labels_FT)
validation_labels_JP = torch.tensor(validation_labels_JP)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels_IE[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels_IE[0])
print(validation_masks[0])

tensor([   101,   9663,  11018,   9672,  11287,   8924,  82838,   9864, 119349,
         32679,  11664,  12092,   9420,  66540,   9960,  40419,  14801,  10739,
          9555,  11664,    113,   9358,  12030,   9434,  45465,  10530,  68599,
         18622,   9523,  16985,  12424,    114,   9638,  36456,   9248,  49742,
         28911,  12092,  62548,   9546,  26737, 119169,  10739,   9555,  12965,
         12424,   9287, 118854,  41850,  48549, 119229,   9651, 118694,   9689,
        118985,  11489,   9708, 119235,   9864, 119349,  32679,    116,   9764,
         50248,   9568, 107693,   9952,  23990,   8987,  53371,   9663,  11102,
        119351,    112,   8987,  17138,  12605, 118867,  11903,    112,   9157,
          9960,  12424,   9265,  44130,    100,   9952,  77884,  48549,    100,
          9568, 119121,    119,    119,    119,    119,   9955,   9460,  68943,
        118671,  48549,    119,    119,    119,    119,    119,    119,    136,
           100,   9069, 118748,  11664, 

In [79]:
print(len(train_labels_IE),len(train_labels_NS),len(train_labels_FT),len(train_labels_JP))

14353 14353 14353 14353


In [80]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data_IE = TensorDataset(train_inputs, train_masks, train_labels_IE)
train_sampler_IE = RandomSampler(train_data_IE)
train_dataloader_IE = DataLoader(train_data_IE, sampler=train_sampler_IE, batch_size=batch_size)

validation_data_IE = TensorDataset(validation_inputs, validation_masks, validation_labels_IE)
validation_sampler_IE = SequentialSampler(validation_data_IE)
validation_dataloader_IE = DataLoader(validation_data_IE, sampler=validation_sampler_IE, batch_size=batch_size)

In [81]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data_NS = TensorDataset(train_inputs, train_masks, train_labels_NS)
train_sampler_NS = RandomSampler(train_data_NS)
train_dataloader_NS = DataLoader(train_data_NS, sampler=train_sampler_NS, batch_size=batch_size)

validation_data_NS = TensorDataset(validation_inputs, validation_masks, validation_labels_NS)
validation_sampler_NS = SequentialSampler(validation_data_NS)
validation_dataloader_NS = DataLoader(validation_data_NS, sampler=validation_sampler_NS, batch_size=batch_size)

In [82]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data_FT = TensorDataset(train_inputs, train_masks, train_labels_FT)
train_sampler_FT = RandomSampler(train_data_FT)
train_dataloader_FT = DataLoader(train_data_FT, sampler=train_sampler_FT, batch_size=batch_size)

validation_data_FT = TensorDataset(validation_inputs, validation_masks, validation_labels_FT)
validation_sampler_FT = SequentialSampler(validation_data_FT)
validation_dataloader_FT = DataLoader(validation_data_FT, sampler=validation_sampler_FT, batch_size=batch_size)

In [83]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data_JP = TensorDataset(train_inputs, train_masks, train_labels_JP)
train_sampler_JP = RandomSampler(train_data_JP)
train_dataloader_JP = DataLoader(train_data_JP, sampler=train_sampler_JP, batch_size=batch_size)

validation_data_JP = TensorDataset(validation_inputs, validation_masks, validation_labels_JP)
validation_sampler_JP = SequentialSampler(validation_data_JP)
validation_dataloader_JP = DataLoader(validation_data_JP, sampler=validation_sampler_JP, batch_size=batch_size)

# 전처리 - 테스트셋

In [84]:
# 리뷰 문장 추출
sentences = strat_test_set['posts']
sentences[:10]

0    저는 INFP이고 . 제 친한친구들은 각각 Esfj Istp Enfj 인데..🤔 유...
1    건강한 엔팁과 불건강한 엔팁은 어떤 차이가 있을까요?​제 생각엔​건강한 엔팁 - 자...
2    엠헬에서 좋은분들 많이 알고친구도 많이 사귀고 싶어요왜냐면 저는 현실에선 외톨이거든...
3    테스트종류는 여러개있잖아요?근데 그럴때마다 적게는 10개에서 많게는 100개이상의 ...
4    한 7년전 지금우리학교는, 신의탑, 와라편의점 이런 웹툰들이 나올 때부터 보기 시작...
5    제가 ENFP인지 INFP인지 잘 모르겠어요중학교때까지만 해도 INFP로만 계속 나...
6    F"저 오늘 술먹을 거에요. 감바스랑 스테이크랑."T"그러세요."F"술먹을거야아아악...
7    저는 어딘지 모르는 거리의 감성 사진입니다ㅋㅋㅋㅋ색깔별로 깔끔하게 정리+알람 뜬거 ...
8    어릴때 부터 주변 어른들에게 많이 들어 왔습니다 남자가 왜 이렇게 운동도 안좋아하고...
9    제가 반찬거리 사러 갔을때 반찬할 오이를 찾으니까 찾아주시면서 말씀하시더라구요직원:...
Name: posts, dtype: object

In [85]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 저는 INFP이고 . 제 친한친구들은 각각 Esfj Istp Enfj 인데..🤔 유형이 잘 맞는 편일까요..¿ 이 세명 모두 저랑 한번도 안싸워봤어서..! 잇팁친구한테 상처 한번 받아본거 빼고... [SEP]',
 '[CLS] 건강한 엔팁과 불건강한 엔팁은 어떤 차이가 있을까요?\u200b제 생각엔\u200b건강한 엔팁 - 자신의 주장이 확고하고 옳다고 생각하지만 상대방의 의견 또한 옳을 수 있고 받아들여야 한다고생각되면 기꺼이 받아들인다.불건강한 엔팁 - 자신의 주장만이 옳고 내 논리적 사고력과 판단력이 남들보다 뛰어나다고 생각하기 때문에오만하고 독단적이다.\u200b이런 게 있지 않을까 싶은데... 성향별 건강-불건강의 차이 같은 것도 어느 정도 정해진 게 있을까요? [SEP]',
 '[CLS] 엠헬에서 좋은분들 많이 알고친구도 많이 사귀고 싶어요왜냐면 저는 현실에선 외톨이거든요..여기서라도 저를 오픈하고마음을 나눌게요 💗 [SEP]',
 '[CLS] 테스트종류는 여러개있잖아요?근데 그럴때마다 적게는 10개에서 많게는 100개이상의 문항을 테스트할때마다 해야되는데 그게 솔직히 귀찮기도 하잖아요.\u200b그래서 앱으로 모든 테스트를 연관시켜서 그중 중복되는 문항을 추려내고 모든 문항을 모아서 하면 한번의 테스트로 여러 테스트의 결과를 볼수있는거죠!\u200b아이디어 | 요새 성격이나 유형 테스트같은게 가장 유행하는데 심지어 그것을 인식한 몇몇기업은 테스트를 만들어서 자연스럽게 자사홍보에 예비소비자가 부담없이 올수있게끔 하죠. (꽃 테스트라던지~ 19금테스트라던지~ 하이틴테스트라던지~ 영화속 부캐테스트라던지~)\u200b기대효과 | 자기자신에 대해 알고싶어하는걸 한번의 테스트로 모든 결과를 볼수있고 또한 그양이 방대하기에 가끔 취미로 볼수있게하는거죠 그리고 몰랐던 테스트까지 같이 보게되니까 새로운 결과를 또 알수있게되고요!\u200b수익모델 | 호스팅 로열티(계약에 따라), 인앱내 광고배너 [SEP]',
 '[CLS] 한 7년전 지금우리학교는,

In [86]:
# 라벨 추출
labels_IE = strat_test_set['I-E'].values
labels_NS = strat_test_set['N-S'].values
labels_FT = strat_test_set['F-T'].values
labels_JP = strat_test_set['J-P'].values

In [87]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 저는 INFP이고 . 제 친한친구들은 각각 Esfj Istp Enfj 인데..🤔 유형이 잘 맞는 편일까요..¿ 이 세명 모두 저랑 한번도 안싸워봤어서..! 잇팁친구한테 상처 한번 받아본거 빼고... [SEP]
['[CLS]', '저', '##는', 'IN', '##F', '##P', '##이고', '.', '제', '친', '##한', '##친', '##구', '##들은', '각각', 'Es', '##f', '##j', 'Ist', '##p', 'En', '##f', '##j', '인', '##데', '.', '.', '[UNK]', '유', '##형', '##이', '잘', '맞', '##는', '편', '##일', '##까', '##요', '.', '.', '¿', '이', '세', '##명', '모두', '저', '##랑', '한', '##번', '##도', '안', '##싸', '##워', '##봤', '##어', '##서', '.', '.', '!', '[UNK]', '상', '##처', '한', '##번', '받아', '##본', '##거', '빼', '##고', '.', '.', '.', '[SEP]']


In [89]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9663,  11018,  36351,  11565,  11127,  54355,    119,
         9672,   9781,  11102,  55358,  17196,  22879,  63042,  10912,
        10575,  10418,  80013,  10410,  10243,  10575,  10418,   9640,
        28911,    119,    119,    100,   9625,  27506,  10739,   9654,
         9256,  11018,   9924,  18392, 118671,  48549,    119,    119,
          224,   9638,   9435,  16758,  29414,   9663,  62200,   9954,
        35465,  12092,   9521, 119089,  69592, 118991,  12965,  12424,
          119,    119,    106,    100,   9414,  60469,   9954,  35465,
        70930,  40419,  41521,   9391,  11664,    119,    119,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [90]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [91]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels_IE = torch.tensor(labels_IE)
test_labels_NS = torch.tensor(labels_NS)
test_labels_FT = torch.tensor(labels_FT)
test_labels_JP = torch.tensor(labels_JP)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels_IE[0])
print(test_masks[0])

tensor([   101,   9663,  11018,  36351,  11565,  11127,  54355,    119,   9672,
          9781,  11102,  55358,  17196,  22879,  63042,  10912,  10575,  10418,
         80013,  10410,  10243,  10575,  10418,   9640,  28911,    119,    119,
           100,   9625,  27506,  10739,   9654,   9256,  11018,   9924,  18392,
        118671,  48549,    119,    119,    224,   9638,   9435,  16758,  29414,
          9663,  62200,   9954,  35465,  12092,   9521, 119089,  69592, 118991,
         12965,  12424,    119,    119,    106,    100,   9414,  60469,   9954,
         35465,  70930,  40419,  41521,   9391,  11664,    119,    119,    119,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [92]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data_IE = TensorDataset(test_inputs, test_masks, test_labels_IE)
test_sampler_IE = RandomSampler(test_data_IE)
test_dataloader_IE = DataLoader(test_data_IE, sampler=test_sampler_IE, batch_size=batch_size)

test_data_NS = TensorDataset(test_inputs, test_masks, test_labels_NS)
test_sampler_NS = RandomSampler(test_data_NS)
test_dataloader_NS = DataLoader(test_data_NS, sampler=test_sampler_NS, batch_size=batch_size)

test_data_FT = TensorDataset(test_inputs, test_masks, test_labels_FT)
test_sampler_FT = RandomSampler(test_data_FT)
test_dataloader_FT = DataLoader(test_data_FT, sampler=test_sampler_FT, batch_size=batch_size)

test_data_JP = TensorDataset(test_inputs, test_masks, test_labels_JP)
test_sampler_JP = RandomSampler(test_data_JP)
test_dataloader_JP = DataLoader(test_data_JP, sampler=test_sampler_JP, batch_size=batch_size)

# 모델 생성

In [56]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [57]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [93]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [94]:
print(len(train_dataloader_IE),len(train_dataloader_NS),
      len(train_dataloader_FT),len(train_dataloader_JP))

449 449 449 449


In [95]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader_IE) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [96]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)  #precision

In [97]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [100]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader_IE):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader_IE), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader_IE)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader_IE:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.40
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.49
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.68
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.43
  Training epcoh took: 0:05:57

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.41
  Training epcoh took: 0:05:59

Running Validation...
  Accuracy: 0.67
  Validation took: 0:00:13

Training complete!


# 테스트셋 평가

In [101]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader_IE):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader_IE), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    125.    Elapsed: 0:00:27.

Accuracy: 0.70
Test took: 0:00:33


## N-S

In [102]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [104]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader_NS):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader_NS), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader_NS)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader_NS:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.80
  Training epcoh took: 0:05:56

Running Validation...
  Accuracy: 0.19
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.79
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.19
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.80
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.19
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.79
  Training epcoh took: 0:05:57

Running Validation...
  Accuracy: 0.19
  Validation took: 0:00:13

Training complete!


In [105]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader_NS):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader_NS), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    125.    Elapsed: 0:00:27.

Accuracy: 0.19
Test took: 0:00:33


# F-T

In [106]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [107]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader_FT):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader_FT), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader_FT)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader_FT:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.72
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.45
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.72
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.45
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.72
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.45
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.72
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.45
  Validation took: 0:00:13

Training complete!


In [108]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader_FT):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader_FT), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    125.    Elapsed: 0:00:27.

Accuracy: 0.46
Test took: 0:00:33


# J-P

In [109]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [110]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader_JP):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader_JP), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader_JP)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader_JP:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.65
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:05:57

Running Validation...
  Accuracy: 0.65
  Validation took: 0:00:13

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:05:57

Running Validation...
  Accuracy: 0.65
  Validation took: 0:00:13

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.66
  Training epcoh took: 0:05:58

Running Validation...
  Accuracy: 0.65
  Validation took: 0:00:13

Training complete!


In [111]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader_JP):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader_JP), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    125.    Elapsed: 0:00:26.

Accuracy: 0.65
Test took: 0:00:33
